# CLASSIFICATION OF LAB SAMPLES

This notebook analyzes the accuracy of the SVM classifier on the training data and on lab samples.
The training data are:
* all genes (not just regulators);
* averaged;
* do not include T1 (since only one observtion)

# TO DO
1. Structure codes for the analyses conducted
   1. Selection of genes
   1. How counts are processed
      1. Reference - T0/biostat, T0/data source, pooled/data source
1. Evaluation of classifiers
   1. Performance on biostat data
   1. Performance on other samples
      1. Time consistency
      1. Replication consistency
      1. Consistency with expectation
1. Analyses for Eliza:
   1. Accuracy with all 11 and the accuracy as you add in more genes (same as what you just sent but with all 11).
   1. Could you test with just the 10 mycobactin genes (Rv2377c-Rv2386c)?
   1. Could you test with just bacterioferritin (Rv3841)? Just Rv2384?
   1. Is there one gene that gives good accuracy
1. compare accuracy with ranom selection of genes
1. Classifier coefficient profile.
   1. Eliza genes - 1-10
   1. Auto selected genes: 5, 10, 20
1. Consider recomputing the training feature vector using pooled

# Preliminaries

## Imports

In [1]:
import init
from common import constants as cn
from common.trinary_data import TrinaryData, REF_TYPE_BIOREACTOR, REF_TYPE_SELF, REF_TYPE_POOLED
from common.data_provider import DataProvider
from common import trinary_data
from common_python.plots import util_plots
from common_python.classifier import classifier_ensemble
from common_python.classifier import classifier_collection
from common_python.util.persister import Persister
from common import transform_data

import collections
import copy
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_val_score
import seaborn as sns

%matplotlib inline

## Constants

In [2]:
os.path.abspath(".")

'/home/ubuntu/xstate/notebooks'

In [3]:
# Files
DIR = os.path.abspath(".")
PERSISTER_PATH = os.path.join(DIR, "classification_of_lab_samples.pcl")
PERSISTER = Persister(PERSISTER_PATH)

In [4]:
# Data preparation constants
T0 = "T0"
POOLED = "pooled"

In [5]:
# Test samples. This is a dictionary (key is the control) of dictionaries (key is the data sample)
def updateSampleDct(sample_dct):
    for name, sample_data in sample_dct.items():
        data_dct = {n: sample_data.__getattribute__(n) for n in trinary_data.SAMPLES}
        sample_dct[name] = data_dct

In [6]:
# Remove the Persister if data has changed
if False:
    PERSISTER.remove()

In [7]:
PERSISTED_VARIABLES = ["PROVIDER", "DATA_DCT", "SAMPLE_DCT", "CLASSIFIER_BASE", "CLASSIFIER_DCT"]
persisted_str = ", ".join(PERSISTED_VARIABLES)
if PERSISTER.isExist():
    stmt = "%s = PERSISTER.get()" % persisted_str
    exec(stmt)
else:
    PROVIDER = DataProvider()
    PROVIDER.do()
    DATA_DCT = {
        T0: TrinaryData(is_regulator=False, is_dropT1=True, is_averaged=True),
        POOLED: TrinaryData(is_regulator=False, is_dropT1=True,
                            is_averaged=True, calcRef=PROVIDER.calcRefPooled)
    }
    # Sample data
    SAMPLE_DCT = {r: trinary_data.getSampleData(ref_type=r, is_regulator=False) for r in [REF_TYPE_BIOREACTOR, REF_TYPE_SELF, REF_TYPE_POOLED]}
    # Classifiers
    CLASSIFIER_BASE = classifier_ensemble.ClassifierEnsemble(
          classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=20, size=100)
    CLASSIFIER_DCT = {k: copy.deepcopy(CLASSIFIER_BASE) for k in DATA_DCT.keys()}
    _ = [c.fit(DF_X_DCT[k], SER_Y_DCT[k]) for k,c in CLASSIFIER_DCT.items()]
    updateSampleDct(SAMPLE_DCT)
    #
    stmt = "PERSISTER.set((%s))" % persisted_str
    exec(stmt)

In [25]:
PROVIDER.df_stage_matrix

,name,color
timepoint,,
T0,Normoxia,b
T1,Normoxia,b
T2,Transition,c
T3,Transition,c
T4,Transition,c
T5,Transition,c
T6,Transition,c
T7,Transition,c
T8,StageII,y


In [21]:
PROVIDER.getStages(list(range(25)))

KeyError: "None of [Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,\n            17, 18, 19, 20, 21, 22, 23, 24],\n           dtype='int64', name='timepoint')] are in the [index]"

In [8]:
# Construct derivative structures    
DF_X_DCT = {k: t.df_X for k,t in DATA_DCT.items()}
DF_X = DF_X_DCT[T0]
SER_Y_DCT = {k: t.ser_y for k,t in DATA_DCT.items()}
SER_Y = SER_Y_DCT[T0]
STAGE_NAMES = np.array(list(cn.STAGE_DCT.values()))
SAMPLE_DATA_DCT = SAMPLE_DCT[REF_TYPE_BIOREACTOR]
CLASSIFIER = CLASSIFIER_DCT[T0]

In [9]:
FULL_RUN = True

In [10]:
# Gene groupings
MYCOBACTIN_GENES = [
"Rv2377c",
"Rv2378c",
"Rv2379c",
"Rv2380c",
"Rv2381c",
"Rv2382c",
"Rv2383c",
"Rv2384",
"Rv2385",
"Rv2386c",
]
BACTERIOFERRITIN_GENES = [
"Rv2341", "Rv3841", 
]
GENE_DCT = {"mycobactin": MYCOBACTIN_GENES,
            "bacterioferritin 1": [BACTERIOFERRITIN_GENES[0]],
            "bacterioferritin 2": [BACTERIOFERRITIN_GENES[1]],
           }
MYCOBACTIN_PLUS_GENES = list(MYCOBACTIN_GENES)
MYCOBACTIN_PLUS_GENES.append("Rv3841")
DF_X_DCT = {k: df[MYCOBACTIN_GENES] for k, df in DF_X_DCT.items()}

## Helper Functions

In [11]:
def getStage(states):
    """
    Provides the name of the stage for the value of a state.
    
    Parameters
    ----------
    states: list-int
    """
    timepoints = []
    for state in states:
        timepoint = [i for i, v in SER_Y_DCT[T0].iteritems() if v == state][0]
        timepoints.append(timepoint)
    return list(PROVIDER.getStages(timepoints))

# TESTS
assert(getStage([0, 1])[0] == "Transition")
print("OK!")

OK!


In [12]:
def sortColumns(df):
    if df is None:
        return None
    columns = list(df.columns)
    columns.sort()
    df.columns = columns
    df = df.sort_index()
    return df

In [18]:
def plotExpressionLevels(df_X, features, ser_y=None, is_plot=True, title=""):
    """
    Heat map of expression levels. Shades states.
    
    Parameters
    ----------
    df_X: DataFrame (feature vector)
    features: list-str
    """
    # Internal constants
    ROTATION = 30
    FONTSIZE = 14
    # Shade replications
    fig, ax = plt.subplots(1, figsize=(20, 5))
    columns = list(set(features).intersection(df_X.columns))
    columns.sort()
    new_df_X = df_X
    sns.heatmap(new_df_X.T, cmap="seismic", ax=ax, vmin=-1, vmax=1)
    # Shade the classes
    if ser_y is not None:
        alphas = [0.0, 0.4]
        alpha_idx = 0
        for idx, val in enumerate(ser_y.values):
            stage = getStage([val])[0]
            if (idx == 0):
                ax.text(idx, 0, "%s" % stage, fontsize=FONTSIZE, rotation=ROTATION)
            elif ser_y.values[idx-1] != val:
                ax.text(idx, 0, "%s" % stage, fontsize=FONTSIZE, rotation=ROTATION)
                alpha_idx = 1 - alpha_idx
            ax.axvspan(idx, idx+1, facecolor='grey', alpha=alphas[alpha_idx])
    # Other plot characteristics
    ax.set_title(title, fontsize=18)
    #
    if is_plot:
        plt.show()
    else:
        plt.close()
    
# Tests
plotExpressionLevels(DF_X, CLASSIFIER.features, SER_Y, is_plot=False)
print("OK!")

OK!


In [15]:
def evaluateClassifierOnInstances(clf, df_X=DF_X_DCT[T0], ser_y=None, class_names=None, is_plot=True, nrow=6, ncol=4, suptitle=""):
    _, axes = plt.subplots(nrow, ncol, figsize=(18,12))
    for irow in range(nrow):
        for icol in range(ncol):
            ax = axes[irow, icol]
            instance_num = irow*ncol + icol + 2
            instance = "T%d" % instance_num
            if instance not in df_X.index:
                break
            ser_X = df_X.loc[instance, :]
            if (icol + 1 == ncol) and (irow==0):
                is_legend = True
            else:
                is_legend = False
            if (irow + 1 == nrow):
                is_xlabel = True
                if icol == 0:
                    is_ylabel = True
                else:
                    is_ylabel = False
            else:
                is_ylabel = False
                is_xlabel = False
            clf.plotFeatureContributions(ser_X, ax=ax,
                    title=instance, true_class=ser_y.loc[instance], is_plot=False, is_legend=is_legend,
                                               class_names=class_names,
                                               is_xlabel=is_xlabel, is_ylabel=is_ylabel)
    plt.suptitle(suptitle)
    if not is_plot:
        plt.close()
    
# Tests
evaluateClassifierOnInstances(CLASSIFIER, ser_y=SER_Y, class_names=STAGE_NAMES, is_plot=False, nrow=2, ncol=2, suptitle="test")
print("OK!")

OK!


In [19]:
def plotSampleData(data_dct, genes=CLASSIFIER_DCT[T0].features, is_plot=True):
    """
    Plots trinary values of data for the specified genes.
    
    Parameters
    ----------
    data_dct: dict
        key: data sample name
        value: feature vector
    genes: list-str
    """
    for name, df_X in data_dct.items():
        plotExpressionLevels(df_X, genes, title=name, is_plot=is_plot)
            
# Tests
plotSampleData(SAMPLE_DCT[REF_TYPE_BIOREACTOR], is_plot=False)
print("OK!")

OK!


In [ ]:
def reportSamplePredictions(data_dct):
    for name, df_X in data_dct.items():
        print("\n%s" % name)
        df_pred = CLASSIFIER.predict(df_X)
        new_columns = [STAGE_NAMES[c] for c in df_pred.columns]
        df_pred.columns = new_columns
        print(df_pred)

In [ ]:
def plotReplicationsOverTime(prediction_df, repl_strs, time_strs, timeFunc, title="", ax=None, is_plot=True):
    """
    Plots the dominate state predicted over time for each replication.
    
    arameter
    ---------
    prediction_df: DataFrame
        column: stage
        row: time and replication
    repl_strs: list-str
    time_strs: list-str
    timeFunc: Function
        input: index value
        output: time portion
    """
    if ax is None:
        _, ax = plt.subplots(1)
    for repl_str in repl_strs:
        indices = list(prediction_df.index)
        bools = [repl_str in i for i in indices]
        if any(bools):
            indices = [i for i in prediction_df.index if repl_str in i]
            y_vals = []
            x_vals = []
            for idx in indices:
                x_vals.append(timeFunc(idx))
                row = prediction_df.loc[idx, :]
                val = row.max()
                y_val = [c for c in row.index if row[c] == val][0]
                y_vals.append(y_val)
            ax.plot(x_vals, y_vals)
            ax.set_ylim(0, len(STAGE_NAMES))
            yticks = ax.get_yticklabels()[0]
            labels = list(STAGE_NAMES)
            ax.set_xticklabels(x_vals, rotation=90)
            ax.set_yticklabels(labels)
    plt.legend(repl_strs)
    plt.title(title)
    if not is_plot:
        plt.close()
            
# Tests
df_X = SAMPLE_DCT[REF_TYPE_BIOREACTOR]["rustad"]
prediction_df = CLASSIFIER_DCT[T0].predict(df_X)
repl_strs = ["rep%d" % n for n in range(4)]
time_strs = ["4hr", "12hr", "1day", "4day", "7day"]
prediction_df = prediction_df[["H37Rv_hypoxia" in i for i in prediction_df.index]]
timeFunc = lambda i: i.split("_")[-2]
plotReplicationsOverTime(prediction_df, repl_strs, time_strs, timeFunc, is_plot=False)
print("OK!")

## Data

### BioReactor

In [ ]:
for ref in DATA_DCT.keys():
    plotExpressionLevels(DF_X_DCT[ref], DF_X_DCT[ref].columns, title="%s" % ref, ser_y=SER_Y_DCT[ref])

In [ ]:
if FULL_RUN:
    for ref in DATA_DCT.keys():
        suptitle = "%s: Acc=%1.2f" % (ref, np.mean(CLASSIFIER_DCT[ref].scores))
        evaluateClassifierOnInstances(CLASSIFIER_DCT[ref], ser_y=DATA_DCT[ref].ser_y, class_names=STAGE_NAMES, nrow=4, ncol=6, suptitle=suptitle)

### Lab Samples

**Observations**
1. Prefer control values that avoid having a constant value of the trinary value over the samples (since such a situation is unhelpful for distinguishing classes).
1. On training data (Bioreactor)
   1. Many genes have the same within class values
   1. Few genes have the same between class values

In [ ]:
# With Bioreactor reference data
plotSampleData(SAMPLE_DCT[REF_TYPE_BIOREACTOR])

In [ ]:
# With pooled reference data
plotSampleData(SAMPLE_DCT[REF_TYPE_POOLED])

# Classification Validations on Training Samples
Classify T2-T25 and see if result is same as original class. Use 5-fold cross validation, where there is a holdout for each class and the selection is random.


In [ ]:
if False:
    accuracy_dct = {}
    ranks = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    ranks = range(1,12)
    for rank in ranks:
        accuracy_dct[rank] = CLASSIFIER.crossValidate(
            NEW_DATA, num_iter=10, num_holdout=1, filter_high_rank=rank, size=100)
    plt.plot(list(accuracy_dct.keys()), list(accuracy_dct.values()))
    plt.ylim([0, 1.1])
    _ = plt.xlabel("No genes in clasifier")
    _ = plt.ylabel("accuracy")

**TODO**: List the genes used for each coordinate.

In [ ]:
CLASSIFIER.plotSVMCoefficients(title="Classifier Coefficients", class_names=STAGE_NAMES)

The above chart displays the values of the SVM coefficients for each of the genes for the classifier used to identify each state. The vertical lines are error bars ($\pm$ 1 standard deviation) for the value of the coefficient over 100 classifiers constructed using 1 holdout per state.

The coefficients are used to identify the stage that best matches a gene expression vector. Given
an expression vector $\mathbf{x} = (x_1, \cdots, x_N)$, where $x_n \in [-1, 1 ]$ is the expression level of gene $x_n$ and
$c_{m,n}$ is the coefficient for the classifier for stage $m$ and gene $n$:
1. $d_m = \sum_n c_{m,n} x_n$
1. Choose stage $m^{\star}$ such that $m^{\star} = arg max_m \{ d_m \}$

In [ ]:
# Deeper look at training data
plotExpressionLevels(DF_X, CLASSIFIER.features, ser_y=SER_Y, title="")

Several observations are of interest:
1. The coefficients for Transition are mostly negative, which means that this stage is about *under-expression*.
In particular, this stage is characterized by an under-expression of the genes Rv2383c, Rv2382c, Rv2386c, and Rv2385.
1. We see that Stage1a is characterized by large positive values of the coefficients for Rv2385, Rv3841, and Rv2384.
Although this makes sense for Rv2385, it is counter intuitive for
Rv2384 and Rv3841.
The rationale here is more complicated.
Rv2384 is not differentially expressed in Stage 1a; however, it is under-expressed in Transition.
So, a positive coefficient for Rv2384 for Stage 1a allows for differentiation from Transition.
A similar tactic seems to be in play for Rv3841 to distinguish Stage1a from Stage II and Resuscitation.
However, this is complicated by the fact that Transition over-expresses Rv3841.
The classifier compensates for this by having a large coefficient for Rv2384.

1. The other stages are characterized by a mixture of over- and under-expression.

# Accuracy of Gene Groups

In [ ]:
def plotAccuracy(genes, title="", selected_stages=None, size=100, is_plot=True, ax=None):
    """
    Plots the accuracy curve for a collection of genes
    
    Parameters
    ----------
    genes: list-str
    title: str
    selected_stages: list-str
        Create a binary classifier for just these tages
    size: int
    is_plot: bol
    """
    if ax is None:
        _, ax = plt.subplots(1)
    accuracy_dct = {}
    #data = mkDataSubset(genes, selected_stages=selected_stages)
    data = DATA_DCT[T0].subsetToStages(selected_stages, genes=genes)
    ranks = range(1, len(genes) + 1)
    for rank in ranks:
        accuracy_dct[rank] = CLASSIFIER.crossValidate(
            data, num_iter=10, num_holdout=1, filter_high_rank=rank, size=size)
    ax.plot(list(accuracy_dct.keys()), list(accuracy_dct.values()), marker="o")
    ax.set_ylim([0, 1.1])
    _ = ax.set_xlabel("No genes in clasifier")
    _ = ax.set_ylabel("accuracy")
    ax.set_title(title)
    if not is_plot:
        plt.close()
    return accuracy_dct
    
# Tests
genes = GENE_DCT["bacterioferritin 1"]
accuracy_dct = plotAccuracy(GENE_DCT["bacterioferritin 1"], title=", ".join(genes), selected_stages="StageII", size=5, is_plot=False)
assert(np.isclose(list(accuracy_dct.values())[0], 0.5))

In [ ]:
if False:
    gene_dct = {g: [g] for g in GENE_DCT["mycobactin"]}
    gene_dct.update(GENE_DCT)
    for name, genes in gene_dct.items():
        _, ax = plt.subplots(1)
        _ = accuracy_dct = plotAccuracy(genes, title=", ".join(genes), selected_stages=["StageII"], ax=ax, size=100)

In [ ]:
if False:
    gene_dct = {g: [g] for g in GENE_DCT["mycobactin"]}
    gene_dct.update(GENE_DCT)
    for name, genes in gene_dct.items():
        _, ax = plt.subplots(1)
        _ = accuracy_dct = plotAccuracy(genes, title=", ".join(genes), ax=ax, size=100)

In [ ]:
# Details for the 10 genes
if False:
    df_X = DF_X[GENE_DCT["mycobactin"]]
    num_genes = range(1, len(df_X.columns) + 1)
    num_genes = [10]
    for num_gene in num_genes:
        clf = classifier_ensemble.ClassifierEnsemble(
                classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=num_gene, size=100)
        clf.fit(df_X, SER_Y)
        evaluateClassifierOnInstances(clf, df_X=df_X, ser_y=SER_Y, class_names=STAGE_NAMES, nrow=4, ncol=6, is_plot=True)

## Evaluation for good performing genes

# Classification of Samples

In [ ]:
CLASSIFIER.features

In [ ]:
CLASSIFIER.predict(SAMPLE_DATA_DCT["AM_MDM"])

In [ ]:
timeFunc = lambda i: i.split("_")[-2]
for ref_type in [REF_TYPE_BIOREACTOR, REF_TYPE_POOLED]:
    _, ax = plt.subplots(1)
    df_X = SAMPLE_DCT[ref_type]["rustad"]
    prediction_df = CLASSIFIER.predict(df_X)
    repl_strs = ["rep%d" % n for n in range(4)]
    time_strs = ["4hr", "12hr", "1day", "4day", "7day"]
    prediction_df = prediction_df[["H37Rv_hypoxia" in i for i in prediction_df.index]]
    title = "rustad: " + ref_type
    plotReplicationsOverTime(prediction_df, repl_strs, time_strs, title=title, timeFunc=timeFunc, ax=ax)

In [ ]:
for ref_type in [REF_TYPE_BIOREACTOR, REF_TYPE_POOLED]:
    df_X = SAMPLE_DCT[ref_type]["galagan"]
    prediction_df = CLASSIFIER.predict(df_X)
    repl_strs = ["rep%d" % n for n in range(1,4)]
    time_strs = ["d%d" % n for n in [1, 2, 3, 5, 7, 8]]
    timeFunc = lambda i: i[0:2]
    title = "galagan: " + ref_type
    _, ax = plt.subplots(1)
    plotReplicationsOverTime(prediction_df, repl_strs, time_strs, timeFunc, title=title, ax=ax)

In [ ]:
reportSamplePredictions(SAMPLE_DCT[REF_TYPE_BIOREACTOR])

In [ ]:
reportSamplePredictions(SAMPLE_DCT[REF_TYPE_POOLED])

# Detailed Analyses

## GSE167232

In [ ]:
plotExpressionLevels(DF_X, CLASSIFIER.features, title="BioReactor", ser_y=SER_Y)
plotExpressionLevels(SAMPLE_DATA_DCT["GSE167232"], CLASSIFIER.features, title="GSE167232")

In [ ]:
plotExpressionLevels(DF_X_DCT[POOLED], CLASSIFIER.features, title="BioReactor", ser_y=SER_Y)
plotExpressionLevels(SAMPLE_DCT[REF_TYPE_POOLED]["GSE167232"], CLASSIFIER.features, title="GSE167232")

In [ ]:
#CLASSIFIER.plotFeatureContributions(df_X.loc["TB_AM1",:], class_names=STAGE_NAMES)
df_predict = CLASSIFIER.predict(SAMPLE_DATA_DCT["GSE167232"])
indexArr = np.reshape(list(df_predict.index), (4,3))
num_row = 4
num_col =3
_, axes = plt.subplots(num_row, num_col, figsize=(20, 12))
for irow in range(4):
    for icol in range(3):
        if icol == 0:
            is_ylabel = True
        else:
            is_ylabel = False
        if icol == num_col - 1:
            is_legend = True
        else:
            is_legend = False
        if irow == num_row - 1:
            is_xlabel = True
        else:
            is_xlabel = False
        instance = indexArr[irow, icol]
        ser_X = SAMPLE_DATA_DCT["GSE167232"].loc[instance, :]
        CLASSIFIER.plotFeatureContributions(ser_X, class_names=STAGE_NAMES, title=instance, ax=axes[irow,icol], is_plot=False,
                                           is_xlabel=is_xlabel, is_ylabel=is_ylabel, is_legend=is_legend)
plt.show()

## Rustad

In [ ]:
# Rustad analysis
df_X = SAMPLE_DATA_DCT["rustad"]
df_pred = CLASSIFIER.predict(df_X)
instances = [i for i in df_pred.index if "H37Rv_hypoxia" in i]
df_pred = df_pred.loc[instances, :]
times = []
replications = []
for instance in instances:
    if "hr" in instance:
        pos = instance.index("hr_")
    else:
        pos = instance.index("ay_")
    last_pos = pos + 2
    times.append(instance[0:last_pos])
    if "10" in instance:
        replications.append(instance[-2:])
    else:
        replications.append(instance[-1])
timeArr = np.array(times)
replicationArr = np.array(replications)
indexArr = list(df_pred.index)
indexArr = np.array(indexArr)
#
state_dct = {}
for time in set(times):
    idxs = [p for p, i in enumerate(indexArr) if time in i]
    replications_sub = replicationArr[idxs]
    times_sub = timeArr[idxs]
    state_dct[time] = []
    for instance in indexArr[idxs]:
        max_val = np.max(df_pred.loc[instance, :].values)
        state = [s for s in df_pred.columns if df_pred.loc[instance, s] == max_val][0]
        state_dct[time].append(STAGE_NAMES[state])

In [ ]:
time_keys = np.array(["H37Rv_hypoxia_4hr", "H37Rv_hypoxia_8hr", "H37Rv_hypoxia_12hr",
                      "H37Rv_hypoxia_1day", "H37Rv_hypoxia_4day", "H37Rv_hypoxia_7day"])
truncated_time_keys = np.array(["4hr", "8hr", "12hr",
                      "1day", "4day", "7day"])
time_idxs = np.array([list(state_dct.keys()).index(k) for k in time_keys])
repl_dct = {}
for repl in [0, 1, 2]:
    repl_dct[repl] = [state_dct[t][repl] for t in time_keys[time_idxs]]

In [ ]:
_, ax = plt.subplots(1)
repl_dct[-1] = ["Transition", "Stage1a", "Stage1b", "StageII", "Resuscitation"]
repl_dct[-1] = repl_dct[0]
repl_dct[-1][0] = "Transition"
repl_dct[-1][1] = "Stage1a"
repl_dct[-1][2] = "Stage1b"
repl_dct[-1][3] = "StageII"
repl_dct[-1][4] = "Resuscitation"
colors = ["red", "blue", "green"]
for repl in range(-1, 3):
    if repl == -1:
        color = "black"
    else:
        color = colors[repl]
    ax.plot(truncated_time_keys, repl_dct[repl], color=color)
plt.legend([0, 1, 2], loc="upper left")
plt.xlabel("time")
plt.ylabel("Stage")

## AM, MDM

In [ ]:
SAMPLE_DATA_DCT["AM_MDM"]

## Galagan

**TODO**: Plot progression of replications over time

# Summary Comments

1. Choice of control
   1. Criteria for choice
1. Evaluating the quality of classification for test samples
   1. Consistency between replications
   1. Consistency in stage progression of replication
1. Detailed summaries for data sets